# 02-B - Feature Engineering Recovery (Otimizado para Baixa Memória)

Este notebook implementa uma versão simplificada e otimizada do feature engineering, projetada para funcionar em ambientes com limitações de memória.

## Estratégia:
1. **Processar apenas dados essenciais**
2. **Usar agregações em chunks**
3. **Salvar resultados intermediários**
4. **Features mínimas mas eficazes**

In [ ]:
import pandas as pd
import numpy as np
import gc
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print('📚 Bibliotecas carregadas - Versão Recovery')
print('💡 Estratégia: Processamento otimizado para baixa memória')

## 1. Carregamento Otimizado

In [ ]:
# Carregar apenas dados de transações (o mais importante)
print('📂 Carregando dados de transações...')
transacoes = pd.read_parquet('../data/part-00000-tid-5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8-4-1-c000.snappy.parquet')

print(f'📊 Transações carregadas: {transacoes.shape}')
print(f'💾 Memória: {transacoes.memory_usage(deep=True).sum() / (1024**2):.1f} MB')

# Renomear e preparar colunas essenciais
dados = transacoes[['internal_store_id', 'internal_product_id', 'transaction_date', 'quantity']].copy()
dados.columns = ['pdv_id', 'produto_id', 'data', 'quantidade']
dados['data'] = pd.to_datetime(dados['data'])

# Liberar memória
del transacoes
gc.collect()

print('✅ Dados preparados e memória otimizada')

## 2. Agregação Semanal Simplificada

In [ ]:
# Criar semanas
dados['semana'] = dados['data'].dt.to_period('W-MON').dt.start_time

print(f'📅 Período: {dados["semana"].min()} até {dados["semana"].max()}')
print(f'📊 Total de semanas: {dados["semana"].nunique()}')

# Agregação semanal
print('🔄 Realizando agregação semanal...')
agregacao_semanal = dados.groupby(['semana', 'pdv_id', 'produto_id']).agg({
    'quantidade': 'sum'
}).reset_index()

print(f'📊 Agregação criada: {agregacao_semanal.shape}')

# Liberar dados originais
del dados
gc.collect()

## 3. Grid Inteligente Simplificado

In [ ]:
# Usar apenas combinações ativas (estratégia que funcionou)
combinacoes_ativas = agregacao_semanal[['pdv_id', 'produto_id']].drop_duplicates()
semanas_unicas = sorted(agregacao_semanal['semana'].unique())

print(f'🎯 Combinações ativas: {len(combinacoes_ativas):,}')
print(f'📅 Semanas: {len(semanas_unicas)}')

# Criar grid em lotes pequenos
batch_size = 5000  # Lotes pequenos
grid_parts = []

for i in range(0, len(combinacoes_ativas), batch_size):
    batch = combinacoes_ativas.iloc[i:i+batch_size]
    print(f'📦 Processando lote {i//batch_size + 1}: {len(batch)} combinações')
    
    # Criar grid para este lote
    batch_grids = []
    for _, row in batch.iterrows():
        grid = pd.DataFrame({
            'semana': semanas_unicas,
            'pdv_id': row['pdv_id'],
            'produto_id': row['produto_id']
        })
        batch_grids.append(grid)
    
    # Concatenar lote
    batch_grid = pd.concat(batch_grids, ignore_index=True)
    
    # Merge com vendas
    batch_grid = batch_grid.merge(agregacao_semanal, on=['semana', 'pdv_id', 'produto_id'], how='left')
    batch_grid['quantidade'] = batch_grid['quantidade'].fillna(0)
    
    grid_parts.append(batch_grid)
    
    # Limpeza
    del batch_grids, batch_grid
    gc.collect()

# Concatenar todas as partes
print('🔗 Concatenando grid completo...')
dados_grid = pd.concat(grid_parts, ignore_index=True)

# Limpeza
del grid_parts, agregacao_semanal
gc.collect()

print(f'✅ Grid criado: {dados_grid.shape}')

## 4. Features Essenciais (Mínimas mas Eficazes)

In [ ]:
# Ordenar dados
dados_grid = dados_grid.sort_values(['pdv_id', 'produto_id', 'semana'])

# Features temporais básicas
dados_grid['mes'] = dados_grid['semana'].dt.month
dados_grid['semana_ano'] = dados_grid['semana'].dt.isocalendar().week

# Features de lag (apenas as mais importantes)
print('⏰ Criando features de lag...')
for lag in [1, 2, 4]:  # Apenas 3 lags mais importantes
    dados_grid[f'quantidade_lag_{lag}'] = dados_grid.groupby(['pdv_id', 'produto_id'])['quantidade'].shift(lag)

# Rolling features (simplificado)
print('📊 Criando rolling features...')
dados_grid['quantidade_media_4w'] = (
    dados_grid.groupby(['pdv_id', 'produto_id'])['quantidade']
    .rolling(window=4, min_periods=1)
    .mean()
    .reset_index(level=[0,1], drop=True)
)

# Features categóricas simples
dados_grid['pdv_hash'] = dados_grid['pdv_id'].astype(str).apply(hash) % 100
dados_grid['produto_hash'] = dados_grid['produto_id'].astype(str).apply(hash) % 100

print('✅ Features essenciais criadas')

## 5. Limpeza Final Simplificada

In [ ]:
# Remover registros sem lag_4 (mais simples)
print(f'📊 Dados antes da limpeza: {dados_grid.shape}')

dados_final = dados_grid[dados_grid['quantidade_lag_4'].notna()].copy()

print(f'📊 Dados após limpeza: {dados_final.shape}')
print(f'💾 Memória final: {dados_final.memory_usage(deep=True).sum() / (1024**2):.1f} MB')

# Verificar distribuição
zeros = (dados_final['quantidade'] == 0).sum()
non_zeros = (dados_final['quantidade'] > 0).sum()
print(f'📈 Zeros: {zeros:,} ({zeros/len(dados_final)*100:.1f}%)')
print(f'📈 Não-zeros: {non_zeros:,} ({non_zeros/len(dados_final)*100:.1f}%)')

## 6. Salvamento Final

In [ ]:
# Salvar dataset final
dados_final.to_csv('../data/dados_features_completo.csv', index=False)
print('💾 Dataset salvo: dados_features_completo.csv')

# Salvar metadados
import pickle

metadata = {
    'data_processamento': pd.Timestamp.now(),
    'total_registros': len(dados_final),
    'total_features': len(dados_final.columns),
    'estrategia': 'Grid Inteligente Simplificado - Recovery',
    'features_criadas': list(dados_final.columns),
    'periodo_treino': f"{dados_final['semana'].min()} a {dados_final['semana'].max()}"
}

with open('../data/feature_engineering_metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)

print('📋 Metadados salvos: feature_engineering_metadata.pkl')

# Mostrar resumo final
print('\n🎉 FEATURE ENGINEERING RECOVERY CONCLUÍDO!')
print('=' * 50)
print(f'📊 Dataset final: {dados_final.shape}')
print(f'🏷️ Features: {list(dados_final.columns)}')
print('🚀 Pronto para modelagem!')